In [2]:
import threading

In [3]:
threading.active_count()

8

In [5]:
threading.current_thread()

<_MainThread(MainThread, started 4576392704)>

In [6]:
threading.enumerate()

[<_MainThread(MainThread, started 4576392704)>,
 <Thread(IOPub, started daemon 123145551536128)>,
 <Heartbeat(Heartbeat, started daemon 123145568325632)>,
 <Thread(Thread-3 (_watch_pipe_fd), started daemon 123145586188288)>,
 <Thread(Thread-4 (_watch_pipe_fd), started daemon 123145602977792)>,
 <ControlThread(Control, started daemon 123145619767296)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145636556800)>,
 <ParentPollerUnix(Thread-2, started daemon 123145653882880)>]

In [7]:
threading.get_ident()

4576392704

In [8]:
threading.get_native_id()

20687762

In [11]:
main = threading.main_thread()
main, main.ident, main.name, main.daemon, main.native_id

(<_MainThread(MainThread, started 4576392704)>,
 4576392704,
 'MainThread',
 False,
 20687762)

In [12]:
threading.current_thread()

<_MainThread(MainThread, started 4576392704)>

In [ ]:
threading.stack_size()

In [25]:
import time


class DataProcessor(threading.Thread):
    def __init__(self, n):
        self.n = n
        super().__init__()
    
    def run(self):
        print(f"{self.name=}, {self.native_id=}, {self.daemon=}")

        for i in range(self.n, -1, -1):
            time.sleep(1)
            print("after sleep", i)


proc = DataProcessor(10)
proc.start()
print("before join", proc.is_alive())

proc.join()
print("after join", proc.is_alive())

self.name='Thread-13', self.native_id=20886615, self.daemon=Falsebefore join
 True
after sleep 10
after sleep 9
after sleep 8
after sleep 7
after sleep 6
after sleep 5
after sleep 4
after sleep 3
after sleep 2
after sleep 1
after sleep 0
after join False


In [28]:
%%time

N = 10 ** 8


def counter(a, b):
    while a < b:
        a += 1

counter(0, N)

CPU times: user 5.34 s, sys: 91.8 ms, total: 5.43 s
Wall time: 6.37 s


In [30]:
%%time

N = 10 ** 8


def counter(a, b):
    while a < b:
        a += 1


th = threading.Thread(target=counter, args=(0, N))
th.start()
th.join()

CPU times: user 4.96 s, sys: 76.5 ms, total: 5.04 s
Wall time: 5.92 s


In [32]:
%%time

N = 10 ** 8
N_THREADS = 10
COUNT_SIZE = N // N_THREADS


def counter(a, b):
    print(f"THREAD: {threading.current_thread().name=}, {a=}, {b=}")
    while a < b:
        a += 1


threads = [
    threading.Thread(
        target=counter,
        args=(i * COUNT_SIZE, (i + 1)* COUNT_SIZE),
        name=f"th_counter_{i}"
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

THREAD: threading.current_thread().name='th_counter_0', a=0, b=10000000THREAD: threading.current_thread().name='th_counter_1', a=10000000, b=20000000

THREAD: threading.current_thread().name='th_counter_2', a=20000000, b=30000000
THREAD: threading.current_thread().name='th_counter_3', a=30000000, b=40000000
THREAD: threading.current_thread().name='th_counter_4', a=40000000, b=50000000THREAD: threading.current_thread().name='th_counter_5', a=50000000, b=60000000

THREAD: threading.current_thread().name='th_counter_6', a=60000000, b=70000000
THREAD: threading.current_thread().name='th_counter_7', a=70000000, b=80000000
THREAD: threading.current_thread().name='th_counter_8', a=80000000, b=90000000
THREAD: threading.current_thread().name='th_counter_9', a=90000000, b=100000000
CPU times: user 5.84 s, sys: 214 ms, total: 6.05 s
Wall time: 6.6 s


In [33]:
from urllib.request import urlopen

In [40]:
%%time

N = 100
URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(urls):
    for url in urls:
        fetch_url(url)


fetch_batch_urls(URLS)

CPU times: user 1.21 s, sys: 122 ms, total: 1.33 s
Wall time: 20.1 s


In [42]:
%%time

N = 100
URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(urls):
    for url in urls:
        fetch_url(url)


th = threading.Thread(target=fetch_batch_urls, args=(URLS,))
th.start()
th.join()

CPU times: user 1.08 s, sys: 110 ms, total: 1.19 s
Wall time: 17.8 s


In [46]:
%%time

N = 100
URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N
N_THREADS = 10
COUNT_SIZE = N // N_THREADS


def fetch_url(url):
    resp = urlopen(url)
    return resp


def fetch_batch_urls(urls):
    print(f"{threading.current_thread().name}, {len(urls)=}")
    for url in urls:
        fetch_url(url)


threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URLS[i * COUNT_SIZE : (i + 1) * COUNT_SIZE],),
        name=f"th_counter_{i}"
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

th_counter_0, len(urls)=10
th_counter_1, len(urls)=10th_counter_2, len(urls)=10

th_counter_3, len(urls)=10
th_counter_4, len(urls)=10
th_counter_5, len(urls)=10
th_counter_6, len(urls)=10th_counter_7, len(urls)=10
th_counter_8, len(urls)=10

th_counter_9, len(urls)=10
CPU times: user 1.26 s, sys: 153 ms, total: 1.42 s
Wall time: 2.56 s


In [49]:
%%time

count = [0]

def count_operations(a, b):
    for i in range(a, b):
        count[0] += 1


count_operations(0, 1000)
print(count)

[1000]
CPU times: user 453 µs, sys: 291 µs, total: 744 µs
Wall time: 544 µs


In [17]:
%%time

N = 1_000
N_THREADS = 50
COUNT_SIZE = N // N_THREADS

count = [0]


def count_operations(a, b):
    for i in range(a, b):
        x = count[0]

        y = x ** 10000
        for _ in range(x):
            a = y

        count[0] = x + 1


threads = [
    threading.Thread(
        target=count_operations,
        args=(i * COUNT_SIZE, (i + 1) * COUNT_SIZE),
        name=f"th_counter_{i}"
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()


count, count[0] == N

CPU times: user 674 ms, sys: 18.8 ms, total: 693 ms
Wall time: 752 ms


([610], False)

In [8]:
import sys

In [12]:
sys.getswitchinterval()

9.999999999999999e-05

In [15]:
sys.setswitchinterval(0.005)

In [20]:
%%time

N = 1_000
N_THREADS = 50
COUNT_SIZE = N // N_THREADS

count = [0]
lock = threading.Lock()


def count_operations(a, b, lock):
    for i in range(a, b):
        lock.acquire()
        
        x = count[0]

        y = x ** 10000
        for _ in range(x):
            a = y

        count[0] = x + 1
        # raise ValueError()
        
        lock.release()


threads = [
    threading.Thread(
        target=count_operations,
        args=(i * COUNT_SIZE, (i + 1) * COUNT_SIZE, lock),
        name=f"th_counter_{i}"
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()


count, count[0] == N

CPU times: user 717 ms, sys: 22 ms, total: 739 ms
Wall time: 780 ms


([1000], True)

In [22]:
from urllib.request import urlopen

In [26]:
%%time

N = 100
URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N
N_THREADS = 10
COUNT_SIZE = N // N_THREADS

sema = threading.Semaphore(4)


def fetch_url(url, sema):
    with sema:
        resp = urlopen(url)
    return resp


def fetch_batch_urls(urls, sema):
    print(f"{threading.current_thread().name}, {len(urls)=}")

    for url in urls:
        fetch_url(url, sema)


threads = [
    threading.Thread(
        target=fetch_batch_urls,
        args=(URLS[i * COUNT_SIZE : (i + 1) * COUNT_SIZE], sema),
        name=f"th_counter_{i}"
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

th_counter_0, len(urls)=10
th_counter_1, len(urls)=10
th_counter_2, len(urls)=10
th_counter_3, len(urls)=10
th_counter_4, len(urls)=10
th_counter_5, len(urls)=10
th_counter_6, len(urls)=10
th_counter_7, len(urls)=10
th_counter_8, len(urls)=10
th_counter_9, len(urls)=10
CPU times: user 1.21 s, sys: 149 ms, total: 1.36 s
Wall time: 6.94 s


In [27]:
import queue

In [33]:
%%time

N = 100
URL = "https://ru.wikipedia.org/wiki/Python"
URLS = [URL] * N
N_THREADS = 10
COUNT_SIZE = N // N_THREADS

que = queue.Queue()


def fetch_url_worker(que):
    cnt = 0
    while True:
        url = que.get()
        if url is None:
            que.put(url)
            print(f"stop fetch_url_worker {threading.current_thread().name=}, {cnt=}")
            break

        cnt += 1
        resp = urlopen(url)


threads = [
    threading.Thread(
        target=fetch_url_worker,
        args=(que,),
        name=f"th_counter_{i}"
    )
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for url in URLS:
    que.put(url)

que.put(None)

for th in threads:
    th.join()

stop fetch_url_worker threading.current_thread().name='th_counter_4', cnt=9
stop fetch_url_worker threading.current_thread().name='th_counter_1', cnt=10
stop fetch_url_worker threading.current_thread().name='th_counter_8', cnt=10
stop fetch_url_worker threading.current_thread().name='th_counter_9', cnt=10
stop fetch_url_worker threading.current_thread().name='th_counter_2', cnt=10
stop fetch_url_worker threading.current_thread().name='th_counter_7', cnt=10stop fetch_url_worker threading.current_thread().name='th_counter_6', cnt=10

stop fetch_url_worker threading.current_thread().name='th_counter_3', cnt=10stop fetch_url_worker threading.current_thread().name='th_counter_5', cnt=11

stop fetch_url_worker threading.current_thread().name='th_counter_0', cnt=10
CPU times: user 1.14 s, sys: 139 ms, total: 1.28 s
Wall time: 2.73 s
